## Apache Spark 101 @ DevFest 2020

This notebook contains the demo script I've live coded on the talk Apache Spark 101 for the DevFest 2020 season.

Minor variations might occur from the live talk, but all the content is here.

It first shows that we can access both `spark` and `sc` (Spark Session and Spark Context respectively) from within the Dataproc's Jupyter notebook natively.

Then it loads the `t8.shakespeare.txt` file from Google Cloud Storage and perform the classic word count algorithm with the RDD API. Later we convert it to DataFrame and register it for Spark SQL. 

The final step in the demo is to load the `stopwords` dataset from nltk and perform two types of joins: the traditional hash join and a broadcast join, and show the different execution plans.

In [ ]:
spark

In [ ]:
sc

In [ ]:
text = sc.textFile("gs://danicat/devfest2020/t8.shakespeare.txt")

In [ ]:
text.take(10)

In [ ]:
words = text.flatMap(lambda line: line.lower().split())

In [ ]:
words.take(10)

In [ ]:
ones = words.map(lambda w: (w,1))

In [ ]:
ones.take(10)

In [ ]:
counts = ones.reduceByKey(lambda x, y: x + y)

In [ ]:
counts.take(10)

In [ ]:
print(counts.toDebugString().decode('utf-8'))

In [ ]:
df = counts.toDF(["word", "count"])

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
sorted = df.orderBy(df["count"].desc())

In [ ]:
sorted.show()

In [ ]:
df.createOrReplaceTempView("wordcount")

In [ ]:
sorted2 = spark.sql("select * from wordcount order by 2 desc")

In [ ]:
sorted2.show()

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
sw = stopwords.words('english')

In [ ]:
type(sw)

In [ ]:
sw[0:10]

In [ ]:
swdf = spark.createDataFrame(sw, ["word"]) # will not work!

In [ ]:
# This is a hack because createDataFrame needs a tuple
swdf = spark.createDataFrame(map(lambda x: (x,), sw), ["word"])

In [ ]:
swdf.show()

In [ ]:
swdf.createOrReplaceTempView("stopwords")

In [ ]:
nostop = spark.sql("select * from wordcount w where not exists (select 1 from stopwords s where s.word = w.word) order by 2 desc")

In [ ]:
nostop.show()

In [ ]:
nostop.explain()

In [ ]:
from pyspark.sql.functions import broadcast # you can also use a sql hint instead

In [ ]:
bcsw = broadcast(swdf)

In [ ]:
bcsw.createOrReplaceTempView("bc_sw")

In [ ]:
bc_nostop = spark.sql("select * from wordcount w where not exists (select 1 from bc_sw b where b.word = w.word) order by 2 desc")

In [ ]:
bc_nostop.show()

In [ ]:
bc_nostop.explain()

In [ ]:
bc_nostop.explain(True)